In [5]:
import pandas as pd

In [6]:
actual_data = {
    "month": [300, 300, 301, 301, 302, 302],
    "country": [1, 2, 1, 2, 1, 2],
    "ln_ged_sb_dep": [0, 1, 1, 2, 2, 3]
}
df_actual = pd.DataFrame(actual_data).set_index(['month', 'country'])
df_actual

ln_ged_sb_dep
month country               
300   1                    0
      2                    1
301   1                    1
      2                    2
302   1                    2
      2                    3

In [7]:
df_pred0 = pd.DataFrame({
    "month": [300, 300, 301, 301],
    "country": [1, 2, 1, 2],
    "value": [1, 2, 3, 4]
}).set_index(['month', 'country'])

df_pred1 = pd.DataFrame({
    "month": [301, 301, 302, 302],
    "country": [1, 2, 1, 2],
    "value": [2, 3, 4, 5]
}).set_index(['month', 'country'])

In [8]:
df_pred0, df_pred1

(               value
 month country       
 300   1            1
       2            2
 301   1            3
       2            4,
                value
 month country       
 301   1            2
       2            3
 302   1            4
       2            5)

In [9]:
pd.concat([df_pred0, df_pred1])

value
month country       
300   1            1
      2            2
301   1            3
      2            4
      1            2
      2            3
302   1            4
      2            5

In [10]:
from metrics import MetricsManager
metrics_list = ['RMSLE', 'CRPS', 'ABC']
metrics_manager = MetricsManager(metrics_list)

In [11]:
metrics_manager.step_wise_evaluation(df_actual, [df_pred0, df_pred1], 'ln_ged_sb_dep', [1, 2])[1]

Metric ABC is not a default metric, skipping...


,RMSLE,CRPS
step01,1.0,1.0
step02,2.0,2.0


In [12]:
metrics_manager.time_series_wise_evaluation(df_actual, [df_pred0, df_pred1], 'ln_ged_sb_dep')[0]

Metric ABC is not a default metric, skipping...


{'ts00': EvaluationMetrics(RMSLE=1.5811388300841898, CRPS=1.5, AP=None, Brier=None, Jeffreys=None, Coverage=None, EMD=None, SD=None, pEMDiv=None, Pearson=None, Variogram=None),
 'ts01': EvaluationMetrics(RMSLE=1.5811388300841898, CRPS=1.5, AP=None, Brier=None, Jeffreys=None, Coverage=None, EMD=None, SD=None, pEMDiv=None, Pearson=None, Variogram=None)}

In [13]:
metrics_manager.month_wise_evaluation(df_actual, [df_pred0, df_pred1], 'ln_ged_sb_dep')[0]

Metric ABC is not a default metric, skipping...


{'month300': EvaluationMetrics(RMSLE=1.0, CRPS=1.0, AP=None, Brier=None, Jeffreys=None, Coverage=None, EMD=None, SD=None, pEMDiv=None, Pearson=None, Variogram=None),
 'month301': EvaluationMetrics(RMSLE=1.5811388300841898, CRPS=1.5, AP=None, Brier=None, Jeffreys=None, Coverage=None, EMD=None, SD=None, pEMDiv=None, Pearson=None, Variogram=None),
 'month302': EvaluationMetrics(RMSLE=2.0, CRPS=2.0, AP=None, Brier=None, Jeffreys=None, Coverage=None, EMD=None, SD=None, pEMDiv=None, Pearson=None, Variogram=None)}

In [35]:
from statistics import mean
def calculate_mean_evaluation_metrics(evaluation_dict) -> dict:
    """
    Calculate the mean evaluation metrics for a dictionary of evaluation metrics.

    Args:
        evaluation_dict (Dict[str, EvaluationMetrics]): A dictionary of evaluation metrics,
            where the keys are time steps, months, or time series, and values are `EvaluationMetrics` instances.

    Returns:
        dict: A dictionary of mean evaluation metrics for the input dictionary.
    """
    mean_dict = {}
    first_item = next(iter(evaluation_dict.values()))
    metric_names = vars(first_item).keys()
    
    # Compute the mean for each metric, skipping metrics with None values
    for key in metric_names:
        valid_values = [value for value in (vars(item).get(key) for item in evaluation_dict.values()) if value is not None]
        if valid_values: 
            mean_dict[key] = mean(valid_values)

    return mean_dict

In [36]:
s = metrics_manager.step_wise_evaluation(df_actual, [df_pred0, df_pred1], 'ln_ged_sb_dep', [1, 2])[0]
s

Metric ABC is not a default metric, skipping...


{'step01': EvaluationMetrics(RMSLE=1.0, CRPS=1.0, AP=None, Brier=None, Jeffreys=None, Coverage=None, EMD=None, SD=None, pEMDiv=None, Pearson=None, Variogram=None),
 'step02': EvaluationMetrics(RMSLE=2.0, CRPS=2.0, AP=None, Brier=None, Jeffreys=None, Coverage=None, EMD=None, SD=None, pEMDiv=None, Pearson=None, Variogram=None)}

In [39]:
calculate_mean_evaluation_metrics(s)

{'RMSLE': 1.5, 'CRPS': 1.5}